<a href="https://colab.research.google.com/github/MSDS-Capstone-Project/LLM_FineTuning/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
### Loading the Dataset

try: from datasets import load_dataset
except:
    !pip install datasets
    from datasets import load_dataset
dataset = load_dataset("nvidia/OpenMathInstruct-2")

print(dataset)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.41k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

...

Generating train_1M split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating train_2M split:   0%|          | 0/2000000 [00:00<?, ? examples/s]

Generating train_5M split:   0%|          | 0/5000000 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/32 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['problem', 'generated_solution', 'expected_answer', 'problem_source'],
        num_rows: 13972791
    })
    train_1M: Dataset({
        features: ['problem', 'generated_solution', 'expected_answer', 'problem_source'],
        num_rows: 1000000
    })
    train_2M: Dataset({
        features: ['problem', 'generated_solution', 'expected_answer', 'problem_source'],
        num_rows: 2000000
    })
    train_5M: Dataset({
        features: ['problem', 'generated_solution', 'expected_answer', 'problem_source'],
        num_rows: 5000000
    })
})


In [12]:
### Logging into HuggingFace

try:
  from dotenv import load_dotenv
except:
  !pip install python-dotenv
  from dotenv import load_dotenv

from huggingface_hub import login, whoami
import os

load_dotenv()
huggingface_api_key = os.getenv("hugging_face_key2")

login(huggingface_api_key)

whoami()


{'type': 'user',
 'id': '670059fac67c24aead74fe1c',
 'name': 'vishgp',
 'fullname': 'Vishwanath Guruvayur',
 'isPro': False,
 'avatarUrl': '/avatars/59f6672f2cd4248af08204eb2ff1d68b.svg',
 'orgs': [],
 'auth': {'type': 'access_token',
  'accessToken': {'displayName': 'LLM_Course',
   'role': 'fineGrained',
   'createdAt': '2025-04-14T13:10:46.259Z',
   'fineGrained': {'canReadGatedRepos': True,
    'global': [],
    'scoped': [{'entity': {'_id': '670059fac67c24aead74fe1c',
       'type': 'user',
       'name': 'vishgp'},
      'permissions': ['collection.read',
       'repo.content.read',
       'repo.write',
       'collection.write']}]}}}}

In [23]:
### SETTING UP THE MODEL

import bitsandbytes as bnb
import torch
import torch.nn as nn
from transformers import BitsAndBytesConfig, AutoTokenizer, AutoModelForCausalLM, AutoConfig

model_name = "gpt2"
# bnb_config = BitsAndBytesConfig(load_in_4bit=True)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                            #  quantization_config = bnb_config,
                                             device_map = "auto"
                                          )
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [24]:
### Freezing the Original Weights

for param in model.parameters():
    param.requires_grad = False # freezing the model - training the adapters later
    if param.ndim == 1:
        # casting the small params to fp32 for more stability
        param.data = param.data.to(torch.float32)


model.gradient_checkpointing_enable()
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [25]:
### LoRA Config

from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,  # Rank of the low-rank adaptation - attention heads
    lora_alpha=32, # alpha scaling
    lora_dropout=0.1,
    bias="none",
    target_modules=["c_attn", "c_proj"]  # GPT-2 specific layers
)


lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()


trainable params: 811,008 || all params: 125,250,816 || trainable%: 0.6475


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [26]:

def format_prompt(example):
    # Combining the problem and expected answer into a single text
    text = f"Problem: {example['problem']}\nSolution: {example['expected_answer']}"

    # Tokenize the combined text
    tokenized = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=512  # Adjust max_length as needed
    )

    # Use the same input_ids for labels
    tokenized["labels"] = tokenized["input_ids"].copy()

    return tokenized

# Loading the dataset
train_data = dataset['train_1M']

# Tokenize the dataset
tokenized_dataset = train_data.map(
    format_prompt,
    batched=True,
    remove_columns=train_data.column_names  # Remove original columns
)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

print(tokenized_dataset[0])

Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

{'input_ids': tensor(40781), 'attention_mask': tensor(1), 'labels': tensor(40781)}


In [32]:
from transformers import TrainingArguments, Trainer

# Training arguments
training_args = TrainingArguments(
    output_dir="./lora_gpt2",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=100,
    max_steps=1000,
    evaluation_strategy="steps",
    save_strategy="steps",
    logging_dir="./logs",
    logging_steps=100,
    learning_rate=2e-3,
    num_train_epochs=3,
    remove_unused_columns=True
)

# Trainer
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset.select(range(10000)),
)

model.config.use_cache = False  # silencing the warnings. need to re-enable for inference
# Start training
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss
100,5.747500,No log
200,5.880500,No log
300,6.017400,No log
400,5.742700,No log
500,5.622800,No log
600,5.779300,No log
700,5.993300,No log
800,5.752300,No log
900,5.931900,No log
1000,5.753800,No log


TrainOutput(global_step=1000, training_loss=5.82215625, metrics={'train_runtime': 294.7035, 'train_samples_per_second': 27.146, 'train_steps_per_second': 3.393, 'total_flos': 4121616384000.0, 'train_loss': 5.82215625, 'epoch': 0.015625})